In [1]:
# Benodigde packages

import json            # Package om .json files in te laden (bvb kolomnamen zijn zo opgeslagen)
import getpass         # Package om een paswoordveldje te genereren.
import mysql.connector # MySQL package
import numpy as np
import os
import pandas as pd    # Populaire package voor data-verwerking
import sys
from basics import*

sys.version_info       # Check python versie, wij veronderstellen 3.6

sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)

In [2]:
def verbind_met_GB(username, hostname, gegevensbanknaam):
    """
    Maak verbinding met een externe gegevensbank
    
    :param  username:          username van de gebruiker, string
    :param  hostname:          naam van de host, string.
                               Dit is in het geval van een lokale server gewoon 'localhost'
    :param  gegevensbanknaam:  naam van de gegevensbank, string.
    :return connection:        connection object, dit is wat teruggeven wordt 
                               door connect() methods van packages die voldoen aan de DB-API
    """
    
    password = getpass.getpass() # Genereer vakje voor wachtwoord in te geven
    
    connection = mysql.connector.connect(host=hostname,
                                         user=username,
                                         passwd=password,
                                         db=gegevensbanknaam)
    return connection


def run_query(connection, query):
    """
    Voer een query uit op een reeds gemaakte connectie, geeft het resultaat van de query terug
    """
    
    # Making a cursor and executing the query
    cursor = connection.cursor()
    cursor.execute(query)
    
    # Collecting the result and casting it in a pd.DataFrame
    res = cursor.fetchall()
    
    return res


def res_to_df(query_result, column_names):
    """
    Giet het resultaat van een uitgevoerde query in een 'pandas dataframe'
    met vooraf gespecifieerde kolomnamen.
    
    Let op: Het resultaat van de query moet dus exact evenveel kolommen bevatten
    als kolomnamen die je meegeeft. Als dit niet het geval is, is dit een indicatie
    dat je oplossing fout is. (Gezien wij de kolomnamen van de oplossing al cadeau doen)
    
    """
    df = pd.DataFrame(query_result, columns=column_names)
    return df

In [3]:
# Dictionary van kolomnamen inladen

filename = os.path.join(os.getcwd(), 'solution', 'all_q_colnam.json')
col_names = json.load(open(filename, 'r'))    

col_names # Inspecteer dictionary

{'query_01': ['tname', 'year', 'HomeRun'],
 'query_02': ['nameFirst', 'nameLast', 'birthYear', 'birthMonth', 'birthDay'],
 'query_03': ['nameFirst', 'nameLast', 'tname'],
 'query_04': ['tname', 'rank', 'W', 'L', 'nameFirst', 'nameLast'],
 'query_05': ['tname'],
 'query_06': ['tname', 'yearID', 'rank', 'W', 'L'],
 'query_07': ['nameLast', 'nameFirst'],
 'query_08': ['birthState',
  'avg_weight',
  'avg_height',
  'avg_HomeRun',
  'avg_Saves'],
 'query_09': ['yearID', 'tname', 'HomeRun'],
 'query_10': ['yearID', 'tname', 'rank', 'Games']}

In [4]:
def query_01(connection, column_names):
    # Bouw je query
    query="""
    SELECT t.name, t.yearID, t.HR
    FROM   Teams as t
    ORDER by t.HR DESC;
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [5]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

········


In [6]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_01'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_01(c, col_names['query_01'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 

,tname,year,HomeRun
0,Seattle Mariners,1997,264
1,Texas Rangers,2005,260
2,Baltimore Orioles,1996,257
3,Toronto Blue Jays,2010,257
4,Baltimore Orioles,2016,253


In [7]:
def query_02(connection, column_names, datum = '1980-01-16'):
    # Bouw je query
    query="""
    SELECT Master.nameFirst, Master.nameLast, 
           Master.birthYear, Master.birthMonth, Master.birthDay
    FROM   Master
    WHERE  Master.debut> {}
    ORDER by Master.nameLast;
    """.format(datum)
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [8]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

········


In [9]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_02'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_02(c, col_names['query_02'], datum = '1980-01-16')

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 

,nameFirst,nameLast,birthYear,birthMonth,birthDay
0,David,Aardsma,1981,12,27
1,Don,Aase,1954,9,8
2,Andy,Abad,1972,8,25
3,Fernando,Abad,1985,12,17
4,Glenn,Abbott,1951,2,16


In [10]:
def query_03(connection, column_names):
    # Bouw je query
    query="""
    SELECT DISTINCT MS.nameFirst, MS.nameLast, t.name
    FROM   Teams as t, Managers as MG, Master as MS
    WHERE  MG.teamID = t.teamID AND MG.plyrMgr = 'N'
           AND MS.playerID = MG.playerID
    ORDER by t.name;
    """
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [11]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

········


In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_03'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_03(c, col_names['query_03'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head()

In [ ]:
def query_04(connection, column_names, datum_x='1980-01-01', datum_y='1980-01-01'):
    # Bouw je query
    query="""
    SELECT DISTINCT t.name, t.Rank, t.W, t.L, MS.nameFirst, MS.nameLast
    FROM   Teams as t, Managers as MG, Master as MS, Halloffame as HF
    WHERE  MS.playerID = MG.playerID AND HF.playerID = MG.playerID
           AND MG.teamID = t.teamID
           AND HF.yearid > {}
           AND MG.yearID < {}
    ORDER by t.name, t.Rank;
    """.format(datum_x, datum_y)
    ####hoe variablen binnen krijgen 
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_04'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_04(c, col_names['query_04'], datum_x='1980-01-01', datum_y='1980-01-01')

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df


In [ ]:
def query_05(connection, column_names):
    # Bouw je query
    query="""
    SELECT DISTINCT t.name
    FROM   Teams as t
    WHERE  (SELECT  COUNT(*)
            FROM    Teams as t, Managers as MG
            WHERE   t.yearID>'1980-01-01'
                    AND MG.teamID = T.teamID AND MG.plyrMgr = 'Y')>=1;
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_05'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_05(c, col_names['query_05'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

In [ ]:
def query_06(connection, column_names, salaris=20000):
    # Bouw je query
    query="""
    SELECT    t.name, t.yearID, t.Rank, t.W, t.L
    FROM      Teams as t, Salaries as S
    WHERE     t.teamID = S.teamID
    GROUP BY  S.teamID
    
    HAVING    MIN(salary)> {}
    
    ORDER BY  t.W
    """.format(salaris)
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_06'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_06(c, col_names['query_06'], salaris=20000)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

In [ ]:
def query_07(connection, column_names):
    # Bouw je query
    query="""
    SELECT DISTINCT MS.nameLast, MS.nameFirst
    FROM  Managers as MG, Master as MS, Awardsmanagers as AM
    
    WHERE MG.playerID = MS.playerID AND MS.playerID = AM.playerID
    GROUP BY AM.playerID
    HAVING COUNT(DISTINCT AM.awardID)=2

    ORDER by MS.nameLast;
    """
    #    AND   AM.awardID LIKE '%SN%'  
    # AND   AM.awardID LIKE '%BW%' 

    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_07'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_07(c, col_names['query_07'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df


In [4]:
def query_08(connection, column_names, jaar=1990, lengte=75):
    # Bouw je query
    query="""
    
    
    SELECT   MS.birthState, AVG(MS.weight), AVG(MS.height),
             AVG(B.HR), AVG(P.SV)
    FROM     Master as MS, Batting as B, Pitching as P,
             Halloffame as HF
                
    WHERE    HF.yearID >= {} AND
             MS.playerID = HF.playerID AND
             MS.playerID = B.playerID AND
             MS.playerID = P.playerID
             
    GROUP BY MS.BirthState
    HAVING AVG(MS.Height)> {}
    
   
    
        
    ORDER BY MS.BirthState;
    """.format(jaar,lengte)
                 
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [5]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

········


In [6]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_08'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_08(c, col_names['query_08'], jaar=1990, lengte=75)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

,birthState,avg_weight,avg_height,avg_HomeRun,avg_Saves
0,,200.4711,75.0462,0.0000,0.9905
1,AK,205.0000,77.0000,0.0000,1.0476
2,ID,185.0000,77.0000,0.4375,0.0000
3,IN,181.2616,75.0646,0.1696,0.3920
4,KS,220.0000,77.0000,0.0000,0.0000
5,LA,205.8793,75.3668,0.0650,15.0788
6,MI,202.9793,75.7599,0.4920,0.9878
7,MN,196.2916,75.0088,0.0149,0.0595
8,MO,199.3150,76.1795,0.0511,3.6421
9,NC,205.6758,75.5637,0.3219,0.5525


In [ ]:
def query_09(connection, column_names, jaar=1975):
    # Bouw je query
    query="""
    SELECT   t.yearID, t.name, t.HR
    FROM     Teams as t
    WHERE    t.yearID = {}
    ORDER BY t.HR DESC
    LIMIT    1,1; 
    
    
     """.format(jaar)
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_09'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_09(c, col_names['query_09'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

In [ ]:
def query_test10(connection, column_names, jaar=1990):
    # Bouw je query
    
    query="""
    SELECT t.yearID, t.name, t.Rank, t.G
    FROM   Teams as t, Appearances as A
    WHERE  t.yearID ={} AND 
           t.teamID = A.teamID 
           
    GROUP BY t.teamID
    HAVING   1>=ANY( SELECT COUNT( AP.playerID) =1
                  FROM   Appearances as A, Teams as t, AwardsPlayers as AP
                  WHERE  t.yearID = {} AND 
                         t.teamID = A.teamID AND
                         AP.playerID = A.playerID)
    ORDER BY t.name, t.Rank;
    """.format(jaar,jaar)
    
    
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_10'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_test10(c, col_names['query_10'], jaar=1990)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 

In [ ]:
def query_10(connection, column_names, jaar=1990):
    # Bouw je query
    
    query="""
    SELECT t.yearID, t.name, t.Rank, t.G
    FROM   Teams as t, Appearances as A
    WHERE  t.teamID = A.teamID AND t.yearID={}
           AND (SELECT  COUNT(DISTINCT AP.playerID)
                FROM    Appearances as A, Awardsplayers as AP, Teams as t
                WHERE   A.teamID = t.teamID AND A.playerID = AP.playerID AND T.yearID= {}
                        AND (SELECT COUNT(*)
                             FROM   Appearances as A, Awardsplayers as AP, Teams as t
                             WHERE  A.teamID = t.teamID AND t.yearID= {}
                             AND    AP.playerID in (SELECT  A.playerID 
                                                     FROM   Appearances as A, Teams as t
                                                     WHERE  A.teamID = t.TeamID)                 
                       )=1)>0;
    ORDER by t.name, t.Rank
    """.format(jaar,jaar,jaar)
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_10'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_10(c, col_names['query_10'], jaar=1990)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 